<a href="https://colab.research.google.com/github/jeonggunlee/CUDATeaching/blob/master/01_cuda_lab/01_simple.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Simple C Programming with Google Colab

- Instructir: Jeong-Gun Lee (School of Software, Hallym University)
- Email: jeonggun.lee@gmail.com
- Web: http://www.onchp.net


You can make a simple C code with the name of **codename.c** through [ %%writefile codename.c ] 

In [60]:
%%writefile prog.c

#include<stdio.h>

int main(void)
{
    printf("Hello World!~~~");
    
    return 0;  // 정상 완료
}

Overwriting prog.c


In [61]:
!ls

hello	   matrixmul.c	pthread   pthread2.c  sample_data
matrixmul  prog.c	pthread2  pthread.c


In [62]:
!gcc --version

gcc (Ubuntu 7.4.0-1ubuntu1~18.04) 7.4.0
Copyright (C) 2017 Free Software Foundation, Inc.
This is free software; see the source for copying conditions.  There is NO
warranty; not even for MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.



In [0]:
!gcc -o hello prog.c

In [64]:
!./hello

Hello World!~~~

*  *  *
## Pthread Exercise
*  *  *



In [65]:
%%writefile pthread.c

/////////////////////////////////////////////////////////
// POSIX pthread example
// adapted from
// https://computing.llnl.gov/tutorials/pthreads/
//
// gcc thread_1.c -o th -pthread
/////////////////////////////////////////////////////////

 #include <pthread.h>
 #include <stdio.h>
 
 #define NUM_THREADS 5

 /////////////////////////////////////////////////////////
 //  print function used as all threads
 /////////////////////////////////////////////////////////
 void *PrintHello(void *threadid)
 {
    int tid;
    tid = (int)threadid;
    printf("Hello World! It's me, thread #%d!\n", tid);
	
    pthread_exit(NULL);
 }

 /////////////////////////////////////////////////////////
 // start the threads and print in MAIN
 /////////////////////////////////////////////////////////
 int main (int argc, char *argv[])
 {
    // array of thread handles
	pthread_t threads[NUM_THREADS];
	// create return code
    int rc;
	// thread counter
    int t;

	// launch the threads and note that the print order 
	// between MAIN and the threads is
	// not deterministic and may change when you SSH in versus
	// use serial console, and from run to run
    for(t=0; t<NUM_THREADS; t++){
       printf("In main: creating thread %d\n", t);
       rc = pthread_create(&threads[t], NULL, PrintHello, (void *)t);
       if (rc){
          printf("ERROR; return code from pthread_create() is %d\n", rc);
       } // if (rc)	
    } //for(t=0; t<NUM_THREADS; t++)
    /* Last thing that main() should do */
    pthread_exit(NULL);
 } // main

Overwriting pthread.c


In [66]:
!gcc -o pthread pthread.c -lpthread

pthread.c: In function ‘PrintHello’:
pthread.c:21:11: warning: cast from pointer to integer of different size [-Wpointer-to-int-cast]
     tid = (int)threadid;
           ^
pthread.c: In function ‘main’:
pthread.c:45:59: warning: cast to pointer from integer of different size [-Wint-to-pointer-cast]
        rc = pthread_create(&threads[t], NULL, PrintHello, (void *)t);
                                                           ^


In [73]:
time !./pthread

In main: creating thread 0
In main: creating thread 1
Hello World! It's me, thread #0!
Hello World! It's me, thread #1!
In main: creating thread 2
In main: creating thread 3
In main: creating thread 4
Hello World! It's me, thread #2!
Hello World! It's me, thread #4!
Hello World! It's me, thread #3!
CPU times: user 15.9 ms, sys: 16.7 ms, total: 32.6 ms
Wall time: 1.41 s


In [68]:
%%writefile matrixmul.c

#include <pthread.h>
#include <stdlib.h>
#include <stdio.h>
 
#define SIZE 1024   // Size by SIZE matrices
int num_thrd;   // number of threads
 
int A[SIZE][SIZE], B[SIZE][SIZE], C[SIZE][SIZE];
 
// initialize a matrix
void init_matrix(int m[SIZE][SIZE])
{
  int i, j, val = 0;
  for (i = 0; i < SIZE; i++)
    for (j = 0; j < SIZE; j++)
      m[i][j] = val++;
}
 
void print_matrix(int m[SIZE][SIZE])
{
  int i, j;
  for (i = 0; i < SIZE; i++) {
    printf("\n\t| ");
    for (j = 0; j < SIZE; j++)
      printf("%2d ", m[i][j]);
    printf("|");
  }
}
 
// thread function: taking "slice" as its argument
void* multiply(void* slice)
{
  int s = (int)slice;   // retrive the slice info
  int from = (s * SIZE)/num_thrd; // note that this 'slicing' works fine
  int to = ((s+1) * SIZE)/num_thrd; // even if SIZE is not divisible by num_thrd
  int i,j,k;
 
  printf("computing slice %d (from row %d to %d)\n", s, from, to-1);
  for (i = from; i < to; i++)
  {  
    for (j = 0; j < SIZE; j++)
    {
      C[i][j] = 0;
      for ( k = 0; k < SIZE; k++)
        C[i][j] += A[i][k]*B[k][j];
    }
  }
  printf("finished slice %d\n", s);
  return 0;
}
 
int main(int argc, char* argv[])
{
  pthread_t* thread;  // pointer to a group of threads
  int i;
 
  if (argc!=2)
  {
    printf("Usage: %s number_of_threads\n",argv[0]);
    exit(-1);
  }
 
  num_thrd = atoi(argv[1]);
  init_matrix(A);
  init_matrix(B);
  thread = (pthread_t*) malloc(num_thrd*sizeof(pthread_t));
 
  // this for loop not entered if threadd number is specified as 1
  for (i = 1; i < num_thrd; i++)
  {
    // creates each thread working on its own slice of i
    if (pthread_create (&thread[i], NULL, multiply, (void*)i) != 0 )
    {
      perror("Can't create thread");
      free(thread);
      exit(-1);
    }
  }
 
  // main thread works on slice 0
  // so everybody is busy
  // main thread does everything if threadd number is specified as 1
  multiply(0);
 
  // main thead waiting for other thread to complete
  for (i = 1; i < num_thrd; i++)
    pthread_join (thread[i], NULL);

/*    
  printf("\n\n");
  print_matrix(A);
  printf("\n\n\t       * \n");
  print_matrix(B);
  printf("\n\n\t       = \n");
  print_matrix(C);
  printf("\n\n");
*/ 
  free(thread);
 
  return 0;
 
}

Overwriting matrixmul.c


In [69]:
!gcc -o matrixmul matrixmul.c -lpthread

matrixmul.c: In function ‘multiply’:
matrixmul.c:34:11: warning: cast from pointer to integer of different size [-Wpointer-to-int-cast]
   int s = (int)slice;   // retrive the slice info
           ^
matrixmul.c: In function ‘main’:
matrixmul.c:73:53: warning: cast to pointer from integer of different size [-Wint-to-pointer-cast]
     if (pthread_create (&thread[i], NULL, multiply, (void*)i) != 0 )
                                                     ^


In [70]:
time !./matrixmul 1

computing slice 0 (from row 0 to 1023)
finished slice 0
CPU times: user 80.4 ms, sys: 36.8 ms, total: 117 ms
Wall time: 15.5 s


In [71]:
time !./matrixmul 2

computing slice 0 (from row 0 to 511)
computing slice 1 (from row 512 to 1023)
^C
CPU times: user 66.6 ms, sys: 23.8 ms, total: 90.4 ms
Wall time: 11.9 s


In [72]:
time !./matrixmul 4

computing slice 1 (from row 256 to 511)
computing slice 2 (from row 512 to 767)
computing slice 0 (from row 0 to 255)
computing slice 3 (from row 768 to 1023)
^C
CPU times: user 48.4 ms, sys: 17.3 ms, total: 65.7 ms
Wall time: 7.86 s
